# Overview

This notebook demonstrates the calibrated district-level weights.

In [17]:
import pandas as pd
import h5py
from policyengine_us import Microsimulation
from us_congressional_districts.calibrate import get_dataset, create_district_metric_matrix, create_target_matrix, get_data_directory

data_folder = get_data_directory()
ages_district = pd.read_csv(
    data_folder / "input" / "demographics" / "age_district.csv"
)

ages_state = pd.read_csv(
    data_folder / "input" / "demographics" / "age_state.csv"
)

ages_national = pd.read_csv(
    data_folder / "input" / "demographics" / "age_national.csv"
)
dataset = get_dataset("cps_2023", 2023)
# the metrics matrix
matrix = create_district_metric_matrix(dataset, ages_district, 2023)
y = create_target_matrix(ages_district)

with h5py.File(
    data_folder / "output" / "congressional_district_weights.h5", "r"
) as f:
    # the weights matrix
    district_weights = f["2022"][:]

districts = pd.read_csv(data_folder / "input" / "geographies" / "districts.csv")

sim = Microsimulation(dataset=dataset)
household_market_income = sim.calculate("household_market_income", 2023).values
household_count_people = sim.calculate("household_count_people", 2023).values

per_capita_income_by_district = (district_weights @ household_market_income) / (district_weights @ household_count_people)

import plotly.express as px

df = districts.copy()
df = df[:435]
df["per_capita_income"] = per_capita_income_by_district

px.scatter(
    df,
    x="x",
    y="y",
    color="per_capita_income",
)